In [ ]:
import twitter
consumer_key = "<Your Consumer Key Here>"
consumer_secret = "<Your Consumer Secret Here>"
access_token = "<Your Access Token Here>"
access_token_secret = "<Your Access Token Secret Here>"
authorization = twitter.OAuth(access_token, access_token_secret, consumer_key, consumer_secret)
t = twitter.Twitter(auth=authorization, retry=True)

import os
data_folder = os.path.join(os.path.expanduser("~"), "Data", "twitter")
output_filename = os.path.join(data_folder, "python_tweets.json")
import json
original_users = []
tweets = []
user_ids = {}
'''搜索包含python的消息，遍历搜索结果：'''
search_results = t.search.tweets(q="python", count=100)['statuses']
for tweet in search_results:
    if 'text' in tweet:
        original_users.append(tweet['user']['screen_name'])
        user_ids[tweet['user']['screen_name']] = tweet['user']['id']
        tweets.append(tweet['text'])

In [ ]:
from sklearn.base import TransformerMixin
from nltk import word_tokenize
class NLTKBOW(TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return [{word: True for word in word_tokenize(document)} for document in X]
'''加载第六章的分类模型'''    
from sklearn.externals import joblib
model_filename = os.path.join(os.path.expanduser("~"), "Models", "twitter", "python_context.pkl")
context_classifier = joblib.load(model_filename)
y_pred = context_classifier.predict(tweets)
relevant_tweets = [tweets[i] for i in range(len(tweets)) if y_pred[i] == 1]
relevant_users = [original_users[i] for i in range(len(tweets)) if y_pred[i] == 1]

In [ ]:
##  调用相应API：friends/ids获取好友信息
import time
def get_friends(t, user_id):
    friends = []
    cursor = -1 
    #当向Twitter请求数据时，不仅返回所需数据，还返回数据类型为整数的游标， Twitter用游标跟踪每一次请求。
    #如果没有更多内容，游标为0；否则，可以使用游标获得下一页数据。开始把游标设置为1，表明是数据的开始：
    while cursor != 0:
        try:
            results = t.friends.ids(user_id=user_id, cursor=cursor, count=5000)
            friends.extend([friend for friend in results['ids']])
            cursor = results['next_cursor']
            if len(friends) >= 10000:
                break
        except TypeError as e:
            if results is None:
                print("You probably reached your API limit, waiting for 5 minutes")
                sys.stdout.flush()
                time.sleep(5*60) # 5 minute wait
            else:
                raise e
        except twitter.TwitterHTTPError as e:
            break
        finally:
            time.sleep(60)
    return friends


In [ ]:
##  构建网络
friends = {}
'''遍历用户名'''
for screen_name in relevant_users:
    '''拿到用户名的ID号'''
    user_id = user_ids[screen_name]
    '''建立朋友字典'''
    friends[user_id] = get_friends(t, user_id)
'''删除没有朋友的ID号'''
friends = {user_id:friends[user_id] for user_id in friends if len(friends[user_id]) > 0}


from collections import defaultdict
def count_friends(friends):
    friend_count = defaultdict(int)
    for friend_list in friends.values():
        for friend in friend_list:
            '''被多少人关注'''
            friend_count[friend] += 1
    return friend_count

friend_count= count_friends(friends)
from operator import itemgetter
'''对关注数排序'''
best_friends = sorted(friend_count.items(), key=itemgetter(1), reverse=True)

'''建立一循环，凑够150个用户的好友数据后，该循环就会结束。遍历best_friends字典（按照在现有用户中的好友数多少排序），
找到还没有获取好友列表的用户，然后获取他的好友列表，更新friends列表。最后，再次查找谁是最受欢迎的'''
while len(friends) < 150:
    for user_id, count in best_friends:
        if user_id not in friends:
            break
    friends[user_id] = get_friends(t, user_id)
    for friend in friends[user_id]:
        friend_count[friend] += 1
    best_friends = sorted(friend_count.items(), key=itemgetter(1), reverse=True)

'''采集数据大约要运行2h，最好把中间结果保存下来，因为有时不得不关闭计算机。使用json库，就可以轻松把好友字典保存到文件里：'''
import json
friends_filename = os.path.join(data_folder, "python_friends.json")
with open(friends_filename, 'w') as outf:
    json.dump(friends, outf)
'''使用json.load函数，从文件中加载数据：'''    
with open(friends_filename) as inf:
    friends = json.load(inf)

In [ ]:
##  创建图
import networkx as nx
G = nx.DiGraph()
'''只将150名核心用户彼此间的好友关系绘制成图像，其他好友关系由于数据量很大难以可视化。把核心用户作为顶点，添加到图中。'''
main_users = friends.keys()
G.add_nodes_from(main_users)
'''接着要创建边。如果第二个用户是第一个用户的好友，那么就在这两个顶点之间建立一条边。遍历所有的核心用户：'''
for user_id in friends:
    for friend in friends[user_id]:
        if friend in main_users:
            G.add_edge(user_id, friend)
            
%matplotlib inline
nx.draw(G)

from matplotlib import pyplot as plt
plt.figure(3,figsize=(20,20))
nx.draw(G, alpha=0.1, edge_color='b')

In [1]:
##  创建用户相似度图
friends = {user: set(friends[user]) for user in friends}
def compute_similarity(friends1, friends2):
    return len(friends1 & friends2) / len(friends1 | friends2) #杰卡德相似系数（Jaccard Similarity）

def create_graph(friends, threshold=0):
    G = nx.Graph()
    for user1 in friends.keys():
        for user2 in friends.keys():
            if user1 == user2:
                continue
                weight = compute_similarity(friends[user1], friends[user2])
            '''画图'''
            if weight >= threshold:
                G.add_node(user1)
                G.add_node(user2)
                G.add_edge(user1, user2, weight=weight)
    return G
G = create_graph(friends)
plt.figure(figsize=(10,10))

IndentationError: expected an indented block (<ipython-input-1-864bcca17f6b>, line 4)

In [ ]:
'''使用spring_layout布局方法：'''
pos = nx.spring_layout(G)
'''使用pos布局方法，确定顶点位置：'''
nx.draw_networkx_nodes(G, pos)
'''接下来，绘制边。遍历图中的每条边，获得其权重：'''
edgewidth = [ d['weight'] for (u,v,d) in G.edges(data=True)]
'''绘制各条边：'''
nx.draw_networkx_edges(G, pos, width=edgewidth)

In [ ]:
##  寻找子图
##  连同分支
G = create_graph(friends, 0.1)
sub_graphs = nx.connected_component_subgraphs(G)
for i, sub_graph in enumerate(sub_graphs):
    n_nodes = len(sub_graph.nodes())
    print("Subgraph {0} has {1} nodes".format(i, n_nodes))
G = create_graph(friends, 0.25)
sub_graphs = nx.connected_component_subgraphs(G)
for i, sub_graph in enumerate(sub_graphs):
    n_nodes = len(sub_graph.nodes())
    print("Subgraph {0} has {1} nodes".format(i, n_nodes))
    
'''可以用不同的颜色把所有连通分支都画出来。因为各连通分支之间没有连接，因此没必要把它们画到一张图中。

sub_graphs是生成器而不是连通分支列表。用nx.number_connected_components找出连通分支的总数；NetworkX无法使用len函数。
'''
sub_graphs = nx.connected_component_subgraphs(G)
n_subgraphs = nx.number_connected_components(G)
fig = plt.figure(figsize=(20, (n_subgraphs * 3)))
for i, sub_graph in enumerate(sub_graphs):
    ax = fig.add_subplot(int(n_subgraphs / 3), 3, i)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    '''绘制顶点和边（用ax参数绘制相应的子图）。绘图之前需要设置好布局：'''
    pos = nx.spring_layout(G)
    nx.draw_networkx_nodes(G, pos, sub_graph.nodes(), ax=ax, node_size=500)
    nx.draw_networkx_edges(G, pos, sub_graph.edges(), ax=ax)

In [ ]:
##  寻找子图
##  优化参数选取准则
def compute_silhouette(threshold, friends):
    G = create_graph(friends, threshold=threshold)
    if len(G.nodes()) < 2: #检查顶点数是否小于2
        return -99
    sub_graphs = nx.connected_component_subgraphs(G)
    if not (2 <= nx.number_connected_components() < len(G.nodes()) - 1): 
        #轮廓系数的定义还要求至少有两个连通分支（才能计算不同簇之间的距离），并且至少其中一个连通分支有两个顶点（计算簇内距离）。
        return -99
    
    '''需要获取标识着顶点被分到哪个连通分支的标签。遍历所有的连通分支，用字典保存顶点及其所属的连通分支'''
    label_dict = {}
    for i, sub_graph in enumerate(sub_graphs):
        for node in sub_graph.nodes():
            label_dict[node] = i
    '''遍历图中所有顶点，依次获取到每个顶点的标签。需要分两步来做，先按一定顺序取到图，再遍历。因为图中顶点没有明确的顺序，
    但是只要没有改动图，顶点会维持现有顺序。这就表明，只要没有改动图，在图上调用.nodes()方法，返回的顶点顺序总是一致的。代码如下：'''
    labels = np.array([label_dict[node] for node in G.nodes()])
    '''轮廓系数函数接收的是距离矩阵，因此，要想办法把图转换为矩阵。首先，使用NetworkX的to_scipy_sparse_matrix函数把图转换为矩阵形式：'''
    X = nx.to_scipy_sparse_matrix(G).todense()
    X = 1 - X
    return silhouette_score(X, labels, metric='precomputed')
def inverted_silhouette(threshold, friends):
    return -compute_silhouette(threshold, friends)
result = minimize(inverted_silhouette, 0.1, args=(friends,))
'''
 inverted_silhouette： 对我们要最小化的函数compute_silhouette进行取反操作，将其变为损失函数。
 0.1：我们一开始猜测阈值为0.1时，函数取到最小值。
 options={'maxiter':10}：只进行10轮迭代（增加迭代次数，效果可能更好，但运行时间也会相应增加）。
 method='nelder-mead'：使用下山单纯形法（Nelder-Mead）优化方法（SciPy提供的优化方法）。
 args=(friends,)：向被优化的函数传入friends字典参数。
'''